In [ ]:
# Executing this cell will:

# Disable all TQDM outputs in stdout.
import os

os.environ["DISABLE_TQDM"] = "True"

# Setup the python logger for the Public API
from osekit import setup_logging

setup_logging()  # Overwrites the default logger to

# Computing a LTAS with the Public API [^download]

[^download]: This notebook can be downloaded as **{nb-download}`example_ltas_public.ipynb`**.

As always in the **Public API**, the first step is to **build the dataset**.

An `Instrument` can be provided to the `Dataset` for the WAV data to be converted in pressure units. This will lead the resulting spectra to be expressed in dB SPL (rather than in dB FS):

In [ ]:
from pathlib import Path

audio_folder = Path(r"_static/sample_audio/timestamped")

from osekit.public_api.dataset import Dataset
from osekit.core_api.instrument import Instrument

dataset = Dataset(
    folder=audio_folder,
    strptime_format="%y%m%d_%H%M%S",
    instrument=Instrument(end_to_end_db=150.0),
)

dataset.build()

The **Public API** `Dataset` is now analyzed and organized:

In [ ]:
print(f"{' DATASET ':#^60}")
print(f"{'Begin:':<30}{str(dataset.origin_dataset.begin):>30}")
print(f"{'End:':<30}{str(dataset.origin_dataset.end):>30}")
print(f"{'Sample rate:':<30}{str(dataset.origin_dataset.sample_rate):>30}\n")

print(f"{' ORIGINAL FILES ':#^60}")
import pandas as pd

pd.DataFrame(
    [
        {
            "Name": f.path.name,
            "Begin": f.begin,
            "End": f.end,
            "Sample Rate": f.sample_rate,
        }
        for f in dataset.origin_files
    ],
).set_index("Name")

Since we will run a spectral analysis, we need to define the FFT parameters:

In [ ]:
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import hamming

sample_rate = 24_000

sft = ShortTimeFFT(
    win=hamming(1024),
    hop=128,  # This will be forced to len(win) if we compute a LTAS
    fs=sample_rate,
)

To **run analyses** in the **Public API**, use the `Analysis` class:

In [ ]:
from osekit.utils.audio_utils import Normalization
from osekit.public_api.analysis import Analysis, AnalysisType

analysis = Analysis(
    analysis_type=AnalysisType.SPECTROGRAM
    | AnalysisType.MATRIX,  # we want to export both the spectrogram and the sx matrix
    nb_ltas_time_bins=3000,  # This will turn the regular spectrum computation in a LTAS
    sample_rate=sample_rate,
    normalization=Normalization.DC_REJECT,  # Removes the DC component
    fft=sft,
    v_lim=(0.0, 150.0),  # Boundaries of the spectrograms
    colormap="viridis",  # Default value
    name="LTAS",
)

Running the analysis will compute the LTAS and save the output files to disk.

In [ ]:
dataset.run_analysis(analysis=analysis)

As for regular spectrum analyses, the output LTAS is stored in a `SpectroDataset` named after `analysis.name`:

In [ ]:
pd.DataFrame(
    [
        {
            "Exported file": list(sd.files)[0].path.name,
            "Begin": sd.begin,
            "End": sd.end,
            "Sample Rate": sd.fft.fs,
        }
        for sd in dataset.get_dataset(analysis.name).data
    ],
).set_index("Exported file")

In [ ]:
# Reset the dataset to get all files back to place.

dataset.reset()